# Query SAP table and join with local data

A first example demonstrates how to join two SAP tables with an external table. We’ll be using the [ABAP Flight Reference Scenario](https://help.sap.com/docs/ABAP_PLATFORM_NEW/fc4c71aa50014fd1b43721701471913d/a9d7c7c140a0408dbc5966c52d156b49.html), specifically joining the `SFLIGHT` and `SPFLI` tables which contain flight and flight schedule details respectively, with an external table `WEATHER` that holds weather information. We will extract flight information and associated temperatures at departure and arrival cities.

## Import DuckDB & load **ERPL** extension
In the next cells we import duckdb. Then we install the ERPL extension and load it into the current DB-session. Via multiple SET-commands we configure the connection to our SAP development system. In our case we use the docker based [ABAP Platform Trial](https://hub.docker.com/r/sapse/abap-platform-trial). The credentials are set by default, details can be found in the documentation of the docker image. 

In [1]:
import duckdb

In [2]:
con = duckdb.connect(config={"allow_unsigned_extensions": "true"})
con.install_extension("./erpl.duckdb_extension");
con.load_extension("erpl");
con.sql("""
SET sap_ashost = 'localhost';
SET sap_sysnr = '00';
SET sap_user = 'DEVELOPER';
SET sap_password = 'Htods70334';
SET sap_client = '001';
SET sap_lang = 'EN';
""");

-- Loading ERPL Trampoline Extension. --
(The purpose of the extension is to extract dependencies and load the ERPL implementation)
ERPL SAP dependencies extracted and saved to /home/jr/.duckdb/extensions/v0.9.2/linux_amd64.
ERPL extension extracted and saved to /home/jr/.duckdb/extensions/v0.9.2/linux_amd64.
ERPL implementation extension installed from /home/jr/.duckdb/extensions/v0.9.2/linux_amd64/erpl_impl.duckdb_extension.
ERPL implementation extension loaded. For instructions how to use it visit https://erpl.io


In [3]:
con.sql("PRAGMA sap_rfc_ping")

┌─────────┐
│   msg   │
│ varchar │
├─────────┤
│ PONG    │
└─────────┘

If the loading of the extension was successful, we can find the exportet functions in the list of `duckdb_functions()`

In [4]:
con.sql("SELECT * FROM duckdb_functions() WHERE function_name LIKE '%sap%';")

┌───────────────┬─────────────┬──────────────────────┬───────────────┬───┬──────────┬──────────────┬─────────┐
│ database_name │ schema_name │    function_name     │ function_type │ … │ internal │ function_oid │ example │
│    varchar    │   varchar   │       varchar        │    varchar    │   │ boolean  │    int64     │ varchar │
├───────────────┼─────────────┼──────────────────────┼───────────────┼───┼──────────┼──────────────┼─────────┤
│ system        │ main        │ sap_read_table       │ table         │ … │ true     │         1415 │ NULL    │
│ system        │ main        │ sap_describe_fields  │ table         │ … │ true     │         1413 │ NULL    │
│ system        │ main        │ sap_show_tables      │ table         │ … │ true     │         1411 │ NULL    │
│ system        │ main        │ sap_rfc_describe_f…  │ table         │ … │ true     │         1409 │ NULL    │
│ system        │ main        │ sap_rfc_search_fun…  │ table         │ … │ true     │         1407 │ NULL    │
│

## Explore the schema of the relevant tables

The ERPL extension provides the method `sap_describe_fields` to explore the data dictionary schema of the respective table. For exploring local data we also can use the `DESCRIBE` command to get the fields of e.g. a CSV-file.

In [6]:
con.sql("SELECT * FROM sap_describe_fields('SFLIGHT');")

┌─────────┬─────────┬────────────┬──────────────────────┬───┬─────────────┬───────────┬───────────┬──────────┐
│   pos   │ is_key  │   field    │         text         │ … │ check_table │ ref_table │ ref_field │ language │
│ varchar │ varchar │  varchar   │       varchar        │   │   varchar   │  varchar  │  varchar  │ varchar  │
├─────────┼─────────┼────────────┼──────────────────────┼───┼─────────────┼───────────┼───────────┼──────────┤
│ 0001    │ X       │ MANDT      │ Client               │ … │ T000        │           │           │ E        │
│ 0002    │ X       │ CARRID     │ Airline Code         │ … │ SCARR       │           │           │ E        │
│ 0003    │ X       │ CONNID     │ Flight Connection …  │ … │ SPFLI       │           │           │ E        │
│ 0004    │ X       │ FLDATE     │ Flight date          │ … │             │           │           │ E        │
│ 0005    │         │ PRICE      │ Airfare              │ … │             │ SFLIGHT   │ CURRENCY  │ E        │
│

In [7]:
con.sql("SELECT * FROM sap_describe_fields('SPFLI');")

┌─────────┬─────────┬───────────┬──────────────────────┬───┬──────────┬─────────────┬───────────┬───────────┬──────────┐
│   pos   │ is_key  │   field   │         text         │ … │ decimals │ check_table │ ref_table │ ref_field │ language │
│ varchar │ varchar │  varchar  │       varchar        │   │ varchar  │   varchar   │  varchar  │  varchar  │ varchar  │
├─────────┼─────────┼───────────┼──────────────────────┼───┼──────────┼─────────────┼───────────┼───────────┼──────────┤
│ 0001    │ X       │ MANDT     │ Client               │ … │ 000000   │ T000        │           │           │ E        │
│ 0002    │ X       │ CARRID    │ Airline Code         │ … │ 000000   │ SCARR       │           │           │ E        │
│ 0003    │ X       │ CONNID    │ Flight Connection …  │ … │ 000000   │             │           │           │ E        │
│ 0004    │         │ COUNTRYFR │ Country Key          │ … │ 000000   │ SGEOCITY    │           │           │ E        │
│ 0005    │         │ CITYFROM  

In [8]:
con.sql("DESCRIBE SELECT * FROM 'WEATHER.csv'")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ FLDATE      │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ COUNTRY     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CITY        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ TEMPERATURE │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ CONDITION   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

## Join tables query

The actual SQL query joins the three tables and performs the following operations:
- Retrieves flight details from `SFLIGHT` using ERPL's `sap_read_table`, aliasing it as `f`.
- Again using ERPL's `sap_read_table` we join `SPFLI` (aliased as `s`) on `MANDT`, `CARRID`, and `CONNID` to get the flight's city of origin and destination.
- Incorporates two instances of an external weather data CSV file, `w_from` and `w_to`, matching on flight date and respective cities' country and name for departure and arrival.
- Rounds the temperature data to one decimal place for readability.
- Orders the results by `CARRIER_ID`, `CONNECTION_ID`, and `FLIGHT_DATE`.
- Limits the output to the first 25 rows for a concise view.

The output of this query will provide a comprehensive view of the flights, including their departure and arrival cities, and the corresponding temperatures, thus offering valuable insights for flight operations analysis.

In [9]:
con.sql("""
SELECT 
  f.CARRID,
  f.CONNID,
  f.FLDATE,
  s.CITYFROM as CITY_FROM,
  ROUND(w_from.TEMPERATURE, 1) as TEMP_FROM,
  s.CITYTO as CITY_TO,
  ROUND(w_to.TEMPERATURE, 1) as TEMP_TO,
  FROM sap_read_table('SFLIGHT') AS f
  JOIN sap_read_table('SPFLI') AS s 
      ON (f.MANDT = s.MANDT AND f.CARRID = s.CARRID AND f.CONNID = s.CONNID)
  JOIN 'WEATHER.csv' AS w_from
      ON (f.FLDATE = w_from.FLDATE AND s.COUNTRYFR = w_from.COUNTRY AND s.CITYFROM = w_from.CITY)
  JOIN 'WEATHER.csv' AS w_to
      ON (f.FLDATE = w_to.FLDATE AND s.COUNTRYTO = w_to.COUNTRY AND s.CITYTO = w_to.CITY)
  ORDER BY 1, 2, 3
  LIMIT 25
""")

┌─────────┬─────────┬────────────┬───────────┬───────────┬──────────┬─────────┐
│ CARRID  │ CONNID  │   FLDATE   │ CITY_FROM │ TEMP_FROM │ CITY_TO  │ TEMP_TO │
│ varchar │ varchar │    date    │  varchar  │  double   │ varchar  │ double  │
├─────────┼─────────┼────────────┼───────────┼───────────┼──────────┼─────────┤
│ LH      │ 0400    │ 2016-11-18 │ FRANKFURT │      11.7 │ NEW YORK │    21.5 │
│ LH      │ 0400    │ 2017-02-06 │ FRANKFURT │      18.5 │ NEW YORK │    13.8 │
│ LH      │ 0400    │ 2017-04-27 │ FRANKFURT │      18.2 │ NEW YORK │    19.9 │
│ LH      │ 0400    │ 2017-04-28 │ FRANKFURT │      22.3 │ NEW YORK │    18.7 │
│ LH      │ 0400    │ 2017-05-28 │ FRANKFURT │      21.5 │ NEW YORK │    23.1 │
│ LH      │ 0400    │ 2017-07-16 │ FRANKFURT │      20.7 │ NEW YORK │    28.1 │
│ LH      │ 0400    │ 2017-10-04 │ FRANKFURT │      13.2 │ NEW YORK │    27.1 │
│ LH      │ 0400    │ 2017-12-23 │ FRANKFURT │      27.6 │ NEW YORK │    19.4 │
└─────────┴─────────┴────────────┴──────

# Call BAPIS directly
In a second example, we call BAPIs of the flight scenario directly via RFC.

### Explore the API of a BAPI
First we search relevant BAPIS stemming from the flights example.

In [51]:
con.sql("SELECT * FROM sap_rfc_search_function(FUNCNAME='BAPI_FLIGHT*') ORDER BY 1")

┌───────────────────────────────┬────────────────────┬─────────┬─────────┬─────────────────────────────┐
│           FUNCNAME            │     GROUPNAME      │  APPL   │  HOST   │            STEXT            │
│            varchar            │      varchar       │ varchar │ varchar │           varchar           │
├───────────────────────────────┼────────────────────┼─────────┼─────────┼─────────────────────────────┤
│ BAPI_FLIGHT_CHECKAVAILIBILITY │ SAPBC_BAPI_SFLIGHT │         │         │ Check flight availability   │
│ BAPI_FLIGHT_GETDETAIL         │ SAPBC_BAPI_SFLIGHT │         │         │ Find details about a flight │
│ BAPI_FLIGHT_GETLIST           │ SAPBC_BAPI_SFLIGHT │         │         │ Find list of flights        │
│ BAPI_FLIGHT_SAVEREPLICA       │ SAPBC_BAPI_SFLIGHT │         │         │ Save replicated flight data │
└───────────────────────────────┴────────────────────┴─────────┴─────────┴─────────────────────────────┘

With `sap_rfc_invoke` (as the function name implies) we can invoke functions and get return values.

In [7]:
con.sql("SELECT * FROM sap_rfc_invoke('BAPI_FLIGHT_GETLIST', path='/FLIGHT_LIST')")

┌───────────┬───────────────────┬───────────┬────────────┬───┬────────────┬───────────────┬─────────┬──────────┐
│ AIRLINEID │      AIRLINE      │ CONNECTID │ FLIGHTDATE │ … │  ARRDATE   │     PRICE     │  CURR   │ CURR_ISO │
│  varchar  │      varchar      │  varchar  │    date    │   │    date    │ decimal(12,4) │ varchar │ varchar  │
├───────────┼───────────────────┼───────────┼────────────┼───┼────────────┼───────────────┼─────────┼──────────┤
│ LH        │ Lufthansa         │ 0400      │ 2016-11-18 │ … │ 2016-11-18 │      666.0000 │ EUR     │ EUR      │
│ LH        │ Lufthansa         │ 0400      │ 2017-02-06 │ … │ 2017-02-06 │      666.0000 │ EUR     │ EUR      │
│ LH        │ Lufthansa         │ 0400      │ 2017-04-27 │ … │ 2017-04-27 │      666.0000 │ EUR     │ EUR      │
│ LH        │ Lufthansa         │ 0400      │ 2017-04-28 │ … │ 2017-04-28 │      666.0000 │ EUR     │ EUR      │
│ LH        │ Lufthansa         │ 0400      │ 2017-05-28 │ … │ 2017-05-28 │      666.0000 │ EUR 

With the method `sap_rfc_describe_function('$BAPI_NAME')` one can explore the API of a RFC function. The method returns four columns:

- `NAME`: Containing the full name of the BAPI, typically this is the same as the argument of the function.
- `IMPORT`: Contains a list with description of all input types. Have a look especially at the required flag. This parameters have to be provided.
- `EXPORT`: Also list with export parameters.
- `CHANGING`: Are so called in/out parameters, which can be input as well as output.
- `TABLES`: This are parameters in form of tables (which are lists of structs in DuckDB). Tables can also have in/out direction.

In the previous example we used the `path` parameter of the invoke function to select the `FLIGHT_LIST` table.

In [52]:
from IPython.display import JSON

desc = con.sql("SELECT * FROM sap_rfc_describe_function('BAPI_FLIGHT_GETLIST')").df()
#display(desc.iloc[0])
display(JSON(desc["IMPORT"][0], root="IMPORT"))
display(JSON(desc["TABLES"][0], root="TABLES"))

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [56]:
con.sql("SELECT * FROM sap_rfc_describe_function('BAPI_FLIGHT_GETDETAIL')").fetchone()[1]

[{'NAME': 'AIRLINEID',
  'TYPE': 'RFCTYPE_CHAR',
  'DIRECTION': 'RFC_IMPORT',
  'LENGTH': 3,
  'DECIMALS': 0,
  'DEFAULTVALUE': '',
  'PARAMETERTEXT': 'Airline Code',
  'OPTIONAL': False},
 {'NAME': 'CONNECTIONID',
  'TYPE': 'RFCTYPE_NUM',
  'DIRECTION': 'RFC_IMPORT',
  'LENGTH': 4,
  'DECIMALS': 0,
  'DEFAULTVALUE': '',
  'PARAMETERTEXT': 'Flight connection code',
  'OPTIONAL': False},
 {'NAME': 'FLIGHTDATE',
  'TYPE': 'RFCTYPE_DATE',
  'DIRECTION': 'RFC_IMPORT',
  'LENGTH': 8,
  'DECIMALS': 0,
  'DEFAULTVALUE': '',
  'PARAMETERTEXT': 'Departure date',
  'OPTIONAL': False}]

In [86]:
res = con.sql("SELECT * FROM sap_rfc_invoke('BAPI_FLIGHT_GETDETAIL', {'AIRLINEID': 'LH', 'CONNECTIONID': '0400', 'FLIGHTDATE': '2016-11-18'::DATE })").df()
res.to_dict(orient="records")[0]

{'ADDITIONAL_INFO': {'FLIGHTTIME': 444,
  'DISTANCE': Decimal('6162.0000'),
  'UNIT': 'KM',
  'UNIT_ISO': 'KMT',
  'PLANETYPE': 'A340-600',
  'FLIGHTTYPE': ''},
 'AVAILIBILITY': {'ECONOMAX': 330,
  'ECONOFREE': 10,
  'BUSINMAX': 30,
  'BUSINFREE': 0,
  'FIRSTMAX': 20,
  'FIRSTFREE': 0},
 'FLIGHT_DATA': {'AIRLINEID': 'LH',
  'AIRLINE': 'Lufthansa',
  'CONNECTID': '0400',
  'FLIGHTDATE': datetime.date(2016, 11, 18),
  'AIRPORTFR': 'FRA',
  'CITYFROM': 'FRANKFURT',
  'AIRPORTTO': 'JFK',
  'CITYTO': 'NEW YORK',
  'DEPTIME': datetime.time(10, 10),
  'ARRTIME': datetime.time(11, 34),
  'ARRDATE': datetime.date(2016, 11, 18),
  'PRICE': Decimal('666.0000'),
  'CURR': 'EUR',
  'CURR_ISO': 'EUR'},
 'EXTENSION_IN': [],
 'EXTENSION_OUT': [],
 'RETURN': [{'TYPE': 'S',
   'ID': 'BC_IBF',
   'NUMBER': '000',
   'MESSAGE': 'Method was executed',
   'LOG_NO': '',
   'LOG_MSG_NO': '000000',
   'MESSAGE_V1': '',
   'MESSAGE_V2': '',
   'MESSAGE_V3': '',
   'MESSAGE_V4': '',
   'PARAMETER': '',
   'ROW':

# Appendix

## Creating artifical weather information
Of course we did not use real weather information (just in case you were confused), but used the following code to create the CSV file.

In [8]:
import numpy.random as npr

df_weather = con.sql("""
SELECT DISTINCT
  f.FLDATE,
  s.COUNTRYFR as COUNTRY,
  s.CITYFROM as CITY,
  FROM sap_read_table('SFLIGHT') as f
  JOIN sap_read_table('SPFLI') as s 
      ON (f.MANDT = s.MANDT AND f.CARRID = s.CARRID AND f.CONNID = s.CONNID)
UNION
SELECT DISTINCT
  f.FLDATE,
  s.COUNTRYTO as COUNTRY,
  s.CITYTO as CITY
  FROM sap_read_table('SFLIGHT') as f
  JOIN sap_read_table('SPFLI') as s 
      ON (f.MANDT = s.MANDT AND f.CARRID = s.CARRID AND f.CONNID = s.CONNID)
""").to_df()

weather_descriptions = [
    "clear sky",
    "few clouds",
    "scattered clouds",
    "broken clouds",
    "shower rain",
    "rain",
    "thunderstorm",
    "snow",
    "mist",
    "thunderstorm with light rain",
    "thunderstorm with rain",
    "thunderstorm with heavy rain",
    "light thunderstorm",
    "thunderstorm",
    "heavy thunderstorm",
    "ragged thunderstorm",
    "thunderstorm with light drizzle",
    "thunderstorm with drizzle",
    "thunderstorm with heavy drizzle",
    "light intensity drizzle",
    "drizzle",
    "heavy intensity drizzle",
    "light intensity drizzle rain",
    "drizzle rain",
    "heavy intensity drizzle rain",
    "shower rain and drizzle",
    "heavy shower rain and drizzle",
    "shower drizzle",
    "light rain",
    "moderate rain",
    "heavy intensity rain",
    "very heavy rain",
    "extreme rain",
    "freezing rain",
    "light intensity shower rain",
    "shower rain",
    "heavy intensity shower rain",
    "ragged shower rain",
    "light snow",
    "snow",
    "heavy snow",
    "sleet",
    "light shower sleet",
    "shower sleet",
    "light rain and snow",
    "rain and snow",
    "light shower snow",
    "shower snow",
    "heavy shower snow"
]

df_weather["TEMPERATURE"] = npr.normal(loc=20., scale=5., size=len(df_weather))
df_weather["CONDITION"] = npr.choice(weather_descriptions, size=len(df_weather))

df_weather.to_csv("./WEATHER.csv", index=False)